# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import sklearn.datasets

In [4]:
data = sklearn.datasets.load_boston()

In [7]:
data.keys()

['data', 'feature_names', 'DESCR', 'target']

In [9]:
X = data['data']
y = data['target']

In [17]:
X_train = X[:- int(X.shape[0] * 0.25)]
y_train = y[:- int(X.shape[0] * 0.25)]

X_test = X[- int(X.shape[0] * 0.25) :]
y_test = y[- int(X.shape[0] * 0.25) :]

In [37]:
def write_answer(name,ans):
    with open('D:\Leon Projects\Python\Obuchenie na razmetkah\Week4\Task2\%sans.txt' %name,'w') as func:
        func.write(str(ans)) 

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

$L(y, z) = (y - z)^2$ <br>
$L'_z(y, z) = 2(z - y) =&gt; (z - y)$

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [19]:
from sklearn.tree import DecisionTreeRegressor

In [26]:
from sklearn.metrics import mean_squared_error

In [43]:
base_algorithms_list = []
coefficients_list = []
for i in range(50):
    tree = DecisionTreeRegressor(max_depth=5,random_state=42)
    coefficients_list = [0.9] * i 
    if i==0:
        tree.fit(X_train,y_train)
    else:
        predict = gbm_predict(X_train)
        s = y_train - predict
        tree.fit(X_train,s)
    base_algorithms_list.append(tree)

In [25]:
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

In [55]:
def gbm_predict(X, base, coefs):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base, coefs)]) for x in X]

In [44]:
pred  = gbm_predict(X_test)

In [46]:
MSE = mean_squared_error(y_test,pred)

In [49]:
RMSE = np.sqrt(MSE)
print RMSE

5.45558333123


In [50]:
write_answer(2,RMSE)

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 9). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [56]:
base_algorithms_list2 = []
coefficients_list2 = []
for i in range(50):
    tree = DecisionTreeRegressor(max_depth=5,random_state=42)
    if i==0:
        tree.fit(X_train,y_train)
    else:
        predict = gbm_predict(X_train, base_algorithms_list2, coefficients_list2)
        s = y_train - predict
        tree.fit(X_train,s)
    base_algorithms_list2.append(tree)
    coefficients_list2.append(0.9 / (1.0 + i))

In [58]:
pred2  = gbm_predict(X_test, base_algorithms_list2, coefficients_list2)
MSE2 = mean_squared_error(y_test,pred2)

RMSE2 = np.sqrt(MSE2)
print RMSE2

write_answer(3,RMSE2)

5.24074258415


## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [59]:
from sklearn.ensemble import GradientBoostingRegressor

In [60]:
write_answer(4,'2 3')

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [61]:
from sklearn.linear_model import LinearRegression

In [62]:
line_reg = LinearRegression()

In [63]:
line_reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [64]:
pred = line_reg.predict(X_test)

In [66]:
MSE3 = mean_squared_error(y_test,pred)

In [70]:
RMSE3 = np.sqrt(MSE3)
print RMSE3

7.84812179648


In [71]:
write_answer(5,RMSE3)